In [ ]:
import os
import re
import glob
import pandas as pd
import csv
import matplotlib.pyplot as plt
import math
import numpy as np
import random

In [ ]:
#filename of all necessary demoghraphic files
ukbb_data_filename = r'ukb46307.tab'
ukbb_withdrawn_ids_filename = r'ukbb_participants_consent_withdrawn.csv'
ukbb_with_civet_ids_filename = r'ukbb_participants_with_civet.tsv'
ukbb_codes_tokeep_filename = r'ukbb_data_field_codes_reduced_numonly.csv'

In [ ]:
#output filenames
ukbb_data_smaller_filename = r'ukb46307_with_civet_temp_reduced.tab'
ukbb_data_final_output = r'ukb46307_with_civet_reduced.tab'
ukbb_data_final_participants = r'ukb46307_with_civet_participant_list.csv'
ukbb_data_final_fieldnames = r'ukb46307_with_civet_field_code_list.csv'
ukbb_data_final_participants_qc_sample = r'ukb46307_with_civet_participant_list_qc_sample.csv'

In [ ]:
additional_fieldnames = ["f.20123.0.0","f.20124.0.0","f.20125.0.0","f.31.0.0"]

In [ ]:
#Create list of all participants who have opted to withdraw from dataset
ukbb_withdrawn_ids = []
with open(ukbb_withdrawn_ids_filename) as file:
    for line in csv.reader(file, delimiter=','):
        ukbb_withdrawn_ids.append(re.sub("[^0-9]", "", line[0]))
#print(ukbb_withdrawn_ids)
ukbb_withdrawn_ids_len = len(ukbb_withdrawn_ids)
print("Withdrawn participants: " + str(ukbb_withdrawn_ids_len))

In [ ]:
#Create a list of all participants who have been run through CIVET
ukbb_with_civet_ids = []
first = True
with open(ukbb_with_civet_ids_filename) as file:
    for line in csv.reader(file, dialect="excel-tab"):
        if not first:
            ukbb_with_civet_ids.append(re.sub("[^0-9]", "", line[0][4:]))
        first = False
#print(ukbb_with_civet_ids)
ukbb_with_civet_ids_len = len(ukbb_with_civet_ids)
print("Participants with CIVET: " + str(ukbb_with_civet_ids_len))

In [ ]:
#Import a list of the field codes to be used
ukbb_field_codes = []
with open(ukbb_codes_tokeep_filename) as file:
    for line in csv.reader(file, delimiter=','):
        ukbb_field_codes.append(re.sub("[^0-9]", "", line[0])) 
ukbb_field_codes_len = len(ukbb_field_codes)
print("Field codes: " + str(ukbb_field_codes_len))
#print(ukbb_field_codes[:5])

In [ ]:
#Return all instances of every field code (every time point)
def get_idx_field_codes(header_line):
    list_codes = [0]
    curr_idx = 0
    for field_name in header_line:
        if field_name in additional_fieldnames:
            list_codes.append(curr_idx)
        else:
            #pattern which will find the third (when MRI was taken) instance of a measurement
            pattern = r"f\.(.*?)\.2"
            #print(field_name)
            code = re.search(pattern, field_name)
            #check that code exists
            if code:
                code = code.group(1)
                if int(code.split('.')[0]) > int(ukbb_field_codes[0]):
                    ukbb_field_codes.pop(0)
        #if can't find the the code at the third instance get first instead
#         else:
#             #print("here")
#             #pattern which will find the first instance of measurement
#             pattern_firstinstance = r"f\.(.*?)\.0"
#             code_firstinstance = re.search(pattern_firstinstance, field_name)
#             if code_firstinstance:
#                 code_firstinstance = code_firstinstance.group(1)
#                 if int(code_firstinstance.split('.')[0]) > int(ukbb_field_codes[0]):
#                     ukbb_field_codes.pop(0)
        if not ukbb_field_codes:
            return list_codes
        #print(code)
        if code == ukbb_field_codes[0]:
            #print()
            list_codes.append(curr_idx)
        curr_idx += 1
    return list_codes

In [ ]:
#l = get_idx_field_codes(['f.eid', 'f.3.0.0', 'f.3.1.0', 'f.3.2.0', 'f.3.3.0', 'f.3.3.0', 'f.4.0.0','f.31.0.0', 'f.31.1.0', 'f.34.0.0'])
#print(l)

In [ ]:
def read_file():
    
    #do some checks
    if ukbb_withdrawn_ids_len != len(ukbb_withdrawn_ids):
        print("Withdrawn Issue")
        return
    if ukbb_with_civet_ids_len != len(ukbb_with_civet_ids):
        print("Civet IDs Issue")
        return
    if ukbb_field_codes_len != len(ukbb_field_codes):
        print("Field Codes Issue")
        return

    tempx = 0
    count_numkept = 0
    first = True
    code_idxs = []
    with open(ukbb_data_smaller_filename, 'w') as ukbb_data_smaller:
        with open(ukbb_data_filename) as tsv:
            for line in csv.reader(tsv, dialect="excel-tab"):
                #write header to new file
                if first:
                    code_idxs = get_idx_field_codes(line)
                    print(len(code_idxs))
                    out_line = [line[i] for i in code_idxs]
                    ukbb_data_smaller.write('\t'.join(out_line) + '\n')
                    first = False
                else:
                    current_id = line[0]
                    #rint(current_id)
                    #if particpant has not withdrawn check if they have civet
                    if current_id != ukbb_withdrawn_ids[0]:
                        #rint("\tnot withdrawn")
                        #if participant has civet add their data to the new sheet
                        if current_id == ukbb_with_civet_ids[0]:
                            #print("\thas civet")
                            out_line = [line[i] for i in code_idxs]
                            ukbb_data_smaller.write('\t'.join(out_line) + '\n')
                            count_numkept += 1
                            ukbb_with_civet_ids.pop(0)
                        elif int(current_id) > int(ukbb_with_civet_ids[0]):
                            print("Missing Participant")
                            print("Current ID " + current_id)
                            print("Civet ID " + ukbb_with_civet_ids[0])
                            ukbb_with_civet_ids.pop(0)
                            #return
                        #if participant doesn't have civet do not keep their data
                    #if participant has withdrawn do not keep any of their data
                    else:
                        ukbb_withdrawn_ids.pop(0)
                    
                    tempx += 1
                    
                    #check more participants still to keep
                    if not ukbb_with_civet_ids:
                        print("No more civet participants")
                        print("Num kept: " + str(count_numkept))
                        print("Num went through: " + str(tempx))
                        return
                     
                    if tempx % 50000 == 0:
                        print(tempx)
                        print(count_numkept)
                    #    return
    #print(tempx)
    print(count_numkept)

In [ ]:
read_file()

In [ ]:
def check_file_contents():
    ix = 0
    with open(ukbb_data_smaller_filename, 'r') as ukbb_data_smaller_check:
        for line in csv.reader(ukbb_data_smaller_check, dialect="excel-tab"):
            print(line)
            print(len(line))
            ix += 1
            if ix >= 1:
                return
check_file_contents()

In [ ]:
#Load the csv into a dataframe to further modify
df = pd.read_csv(ukbb_data_smaller_filename, sep='\t', header=0)
print(df.shape)

In [ ]:
#Blood Pressure (bp) was taken twice in a row either manually or automatically (not both)
#Create new empty bp cols
df['bp.0'] = np.nan
df['bp.1'] = np.nan
#If bp was taken manually add that measure to new col
df.loc[df['f.94.2.0'].notna(), 'bp.0'] = df['f.94.2.0']
df.loc[df['f.94.2.1'].notna(), 'bp.1'] = df['f.94.2.1']
#If bp was taken automatically add that measure to new col
df.loc[df['f.4079.2.0'].notna(), 'bp.0'] = df['f.4079.2.0']
df.loc[df['f.4079.2.1'].notna(), 'bp.1'] = df['f.4079.2.1']
#Remove old blood pressure cols
df = df.drop(['f.94.2.0','f.4079.2.0','f.94.2.1','f.4079.2.1'],axis=1)

In [ ]:
#Count the number of missing values for each subject and append to new col
df['countnull'] = df.isna().sum(axis=1)
#df['countnull']

In [ ]:
df_countnulls = df['countnull'].to_numpy()

In [ ]:
seriesObj = df.apply(lambda df: True if df['countnull'] <= 15 else False , axis=1)
# Count number of True in series
numOfRows = len(seriesObj[seriesObj == True].index)
print('Number of Participants whop have at least 40 out of 55 variables: ', numOfRows)

In [ ]:
#Graph the number of participants with data availablity
#plt.plot(bins, count_aoc)
#plt.title('Number of Pariticpants with at least _% data avaiable')
#plt.xlabel('% Data Available')
#plt.ylabel('# Participants')
#plt.show()

In [ ]:
num_participants_before = df.shape[0]
#Remove pariticpants who are missing more than 15 variables
df = df.drop(df[df['countnull'] > 15].index)
num_participants_after = df.shape[0]
#Print how many participants were removed
print(str(num_participants_before-num_participants_after) + " removed")

In [ ]:
halfp = df.shape[0] // 2
eightyp = math.trunc(df.shape[0] * .8)
ninteyp = math.trunc(df.shape[0] * .9)
fifteenp = math.trunc(df.shape[0] * .15)
halfmissing = []
eightymissing = []
ninteymissing = []
fifteenmissing = []
for col in df.columns:
    if df[col].isna().sum() >= halfp:
        halfmissing.append(col)
    if df[col].isna().sum() >= eightyp:
        eightymissing.append(col)
    if df[col].isna().sum() >= ninteyp:
        ninteymissing.append(col)

In [ ]:
print(len(halfmissing))
print(len(eightymissing))
print(len(ninteymissing))

In [ ]:
print(ninteymissing)

In [ ]:
df = df.drop(ninteymissing,axis=1)
df = df.drop('countnull',axis=1)

In [ ]:
print(df.shape)

In [ ]:
df

In [ ]:
#export to csv
df.to_csv(ukbb_data_final_output, index=False, sep='\t')
#Export list of all participants to separate csv
df_participants = df[['f.eid','f.31.0.0']]
df_participants.to_csv(ukbb_data_final_participants, index=False)
#Export list of all variables to separate csv
df.head(0).to_csv(ukbb_data_final_fieldnames, index=False)

In [ ]:
fig, ax = plt.subplots()
ax.bar(tt.keys(),tt.values)#color=['blue', 'cyan'])
#    plt.text(value, index, str(value))

ax.text(0.05, 0.95, "na = " + str(nacount), transform=ax.transAxes, fontsize=14,
        verticalalignment='top')
#ax.set_xlim(0,1.1)
#ax.set_ylabel("# Participants")
ax.set_xlabel("Blood Pressure")
plt.show()